In [ ]:
%matplotlib notebook
#%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import math
import h5py
import argparse
import importlib
import data_utils
import numpy as np
import tensorflow as tf
from datetime import datetime


In [ ]:

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation
import mayavi.mlab as mlab
import scipy.spatial as spatial
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection

from matplotlib import animation

import numpy as np

In [ ]:
point_size=0.2
axes_str = ["X", "Y", "Z"]

color_map = ["#CFCDC9", "b","r","g"]
axes_limits = [
     [-100, 80], # X axis range
     [-10, 10], # Y axis range
     [-3, 10]   # Z axis range
 ] 

def draw_box(pyplot_axis, vertices, axes=[0, 1, 2], color='black'):

    
    connections = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # Lower plane parallel to Z=0 plane
        [4, 5], [5, 6], [6, 7], [7, 4],  # Upper plane parallel to Z=0 plane
        [0, 4], [1, 5], [2, 6], [3, 7]  # Connections between upper and lower planes
    ]
    for connection in connections:
        plt.plot(vertices[ connection[0], :],vertices[ connection[1], :], c=color, lw=0.5)

        
def draw_point_cloud(ax, title, axes=[0, 1, 2], xlim3d=None, ylim3d=None, zlim3d=None):
    
    class_color =  [color_map[int(i)] for i in velo_frame[:, 3]] 
    point_sizes =  [ 0.01 if i < 1 else point_size for i in velo_frame[:, 3]] 

    ax.scatter(*np.transpose(velo_frame[:, axes]), s=point_sizes, c=class_color, cmap='gray')
    ax.set_title(title)
    ax.set_xlabel('{} axis'.format(axes_str[axes[0]]))
    ax.set_ylabel('{} axis'.format(axes_str[axes[1]]))
   
    if len(axes) > 2:
        ax.set_xlim3d(*axes_limits[axes[0]])
        ax.set_ylim3d(*axes_limits[axes[1]])
        ax.set_zlim3d(*axes_limits[axes[2]])
        ax.set_zlabel('{} axis'.format(axes_str[axes[2]]))
    else:
        ax.set_xlim(*axes_limits[axes[0]])
        ax.set_ylim(*axes_limits[axes[1]])
    if xlim3d!=None:
        ax.set_xlim3d(xlim3d)
    if ylim3d!=None:
        ax.set_ylim3d(ylim3d)
    if zlim3d!=None:
        ax.set_zlim3d(zlim3d)





In [ ]:
filelist = "/data/dataset/merged-10000+non-normalized+block-25/./val/val_half_1_pred.h5"
data = h5py.File(filelist)
data_train = data['data_num'][:,:,:].astype(np.float32)

confidence = data['confidence'][...].astype(np.int64)
label_train = data['label_seg'][...].astype(np.int64)



In [ ]:
filelist = "../pointcnn-dataset/with-reflectance+merged-10000+non-normalized+block-100/val/val_zero_3.h5"
data = h5py.File(filelist)
data_train = data['data'][:,:,:].astype(np.float32)

label = data['label'][...].astype(np.int64)
label_train = data['label_seg'][...].astype(np.int64)
data_num_train = data['data_num'][...].astype(np.int32)
print(data_num_train.shape, label[:19])

In [ ]:
##Check duplicate rate
velo_frame = np.concatenate(data_train[:19])
labels = np.concatenate(label_train[:19])


In [ ]:
data_train[1].shape

In [ ]:
indices_frame = np.nonzero(data_train[:20,:,-1]==2345)

In [ ]:
data_train[indices_frame].shape

In [ ]:
labels[:20][indices_frame]

In [ ]:
np.unique(data_train[indices_frame[0]], return_counts=True)

In [ ]:
id_frame =6
#labels, velo_frame = label_train[id_frame, :data_num_train[id_frame]], data_train[id_frame, :data_num_train[id_frame],:]
labels = np.expand_dims(labels[indices_frame], axis=1)
print(velo_frame.shape, labels.shape)
pointSize = velo_frame.shape[0]
# X, Y, Z, Class, Object_id
velo_frame = np.concatenate( (velo_frame[:,0:1],velo_frame[:,2:3], velo_frame[:,1:2],labels , velo_frame[:,3:4]) , axis=1)
obj_id = 1

velo_frame = velo_frame[ np.logical_and(velo_frame[:,3] >= 1, velo_frame[:,0] >= -100)  ] #load_data#[ load_data[:,3] >= 1 ] #np.logical_and(load_data[:,3] >= 1,  load_data[:,-1]==30) ]
#velo_frame = np.append(velo_frame, [np.amax(velo_frame, axis=0), np.amin(velo_frame, axis=0)], axis=0)

print(np.amax(velo_frame, axis=0))
print(np.amin(velo_frame, axis=0))
f2 = plt.figure(figsize=(10,4))
axes=[0, 1, 2]
ax2 = f2.add_subplot(111, projection='3d')   
order_ids = np.arange(8)
draw_point_cloud(ax2, 'Velodyne scan', xlim3d=None)
plt.show()